<a href="https://colab.research.google.com/github/roshini-joga/3-Layer-Deep-Neural-Network-for-Non-Linear-Regression/blob/master/Pytorch_3layer_Nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 36.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pytorch_lightning as pl
import numpy as np
import matplotlib.pyplot as plt

# Generate synthetic data based on a 3-variable non-linear equation
def generate_data(n=1000):
    x1 = np.random.uniform(-1, 1, n)
    x2 = np.random.uniform(-1, 1, n)
    x3 = np.random.uniform(-1, 1, n)
    y = np.sin(x1) + np.cos(x2) + x3**2  # Non-linear equation
    return np.stack([x1, x2, x3], axis=1), y.reshape(-1, 1)

X, y = generate_data()
X_train, y_train = torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# 1. PyTorch Implementation WITHOUT Built-in Layers


In [ ]:
class SimpleNN_Scratch:
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.weights1 = torch.randn(input_dim, hidden_dim, requires_grad=True)
        self.bias1 = torch.randn(hidden_dim, requires_grad=True)
        self.weights2 = torch.randn(hidden_dim, hidden_dim, requires_grad=True)
        self.bias2 = torch.randn(hidden_dim, requires_grad=True)
        self.weights3 = torch.randn(hidden_dim, output_dim, requires_grad=True)
        self.bias3 = torch.randn(output_dim, requires_grad=True)

    def forward(self, x):
        x = torch.tanh(x @ self.weights1 + self.bias1)
        x = torch.tanh(x @ self.weights2 + self.bias2)
        x = x @ self.weights3 + self.bias3
        return x

    def train(self, X, y, epochs=1000, lr=0.01):
        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = ((y_pred - y) ** 2).mean()
            loss.backward()
            with torch.no_grad():
                self.weights1 -= lr * self.weights1.grad
                self.bias1 -= lr * self.bias1.grad
                self.weights2 -= lr * self.weights2.grad
                self.bias2 -= lr * self.bias2.grad
                self.weights3 -= lr * self.weights3.grad
                self.bias3 -= lr * self.bias3.grad
                self.weights1.grad.zero_()
                self.bias1.grad.zero_()
                self.weights2.grad.zero_()
                self.bias2.grad.zero_()
                self.weights3.grad.zero_()
                self.bias3.grad.zero_()
            if epoch % 100 == 0:
                print(f'Epoch {epoch}, Loss: {loss.item()}')

# Train the scratch implementation
nn_scratch = SimpleNN_Scratch(3, 10, 1)
nn_scratch.train(X_train, y_train)

Epoch 0, Loss: 6.667201995849609
Epoch 100, Loss: 0.0753619596362114
Epoch 200, Loss: 0.049569059163331985
Epoch 300, Loss: 0.03889014571905136
Epoch 400, Loss: 0.03333795815706253
Epoch 500, Loss: 0.029940219596028328
Epoch 600, Loss: 0.027578143402934074
Epoch 700, Loss: 0.025771142914891243
Epoch 800, Loss: 0.024294022470712662
Epoch 900, Loss: 0.02303302474319935


# 2. PyTorch Class-Based Model

In [ ]:

class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.layer2 = nn.Linear(hidden_dim, hidden_dim)
        self.layer3 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = torch.tanh(self.layer1(x))
        x = torch.tanh(self.layer2(x))
        x = self.layer3(x)
        return x

# Train PyTorch Class-Based Model
model = SimpleNN(3, 10, 1)
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

def train_model(model, X, y, epochs=1000):
    for epoch in range(epochs):
        optimizer.zero_grad()
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()
        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

train_model(model, X_train, y_train)

Epoch 0, Loss: 1.3360215425491333
Epoch 100, Loss: 0.07847139984369278
Epoch 200, Loss: 0.0037288612220436335
Epoch 300, Loss: 0.0017127440078184009
Epoch 400, Loss: 0.0010888137621805072
Epoch 500, Loss: 0.0006845139432698488
Epoch 600, Loss: 0.0003728700685314834
Epoch 700, Loss: 0.0001910095161292702
Epoch 800, Loss: 0.00011816297774203122
Epoch 900, Loss: 8.750471170060337e-05


# 3. PyTorch Lightning Implementation


In [ ]:
class LightningNN(pl.LightningModule):
    def __init__(self, input_dim=3, hidden_dim=10, output_dim=1):
        super(LightningNN, self).__init__()
        self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.layer2 = nn.Linear(hidden_dim, hidden_dim)
        self.layer3 = nn.Linear(hidden_dim, output_dim)
        self.loss_fn = nn.MSELoss()

    def forward(self, x):
        x = torch.tanh(self.layer1(x))
        x = torch.tanh(self.layer2(x))
        x = self.layer3(x)
        return x

    def training_step(self, batch, batch_idx):
        X, y = batch
        y_pred = self(X)
        loss = self.loss_fn(y_pred, y)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.01)

# Train PyTorch Lightning Model
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
model_lightning = LightningNN()
trainer = pl.Trainer(max_epochs=1000)
trainer.fit(model_lightning, dataloader)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type    | Params | Mode 
--------------------------------------------
0 | layer1  | Linear  | 40     | train
1 | layer2  | Linear  | 110    | train
2 | layer3  | Linear  | 11     | train
3 | loss_fn | MSELoss | 0      | train
--------------------------------------------
161       Trainable params
0         Non-trainable params
161       Total params
0.001     Total estimated model params size (MB)
4         Modules in train mode
0         Modules in eval mode
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (32) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you wa

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1000` reached.
